In [2]:
import numpy as np
import plotly.graph_objects as go

In [3]:
def filter(b, a, x):
    return np.convolve(x, b, mode='same') - np.convolve(x, a, mode='same')

In [11]:
# Initialization of variables
x1 = 0
x2 = 0
wc1 = 2 * np.pi * 90  # Angular frequency for first signal
wc2 = 2 * np.pi * 100  # Angular frequency for second signal
cost = (x1 - 5) ** 2 + (x2 - 3) ** 2  # Initial cost function
diff = 2 * (x1 - 5) + 2 * (x2 - 3)  # Initial difference calculation
x = []
y = [cost]
t = np.linspace(-np.pi, np.pi, 100)  # Time vector

In [12]:

grad1 = []  # Gradient 1 values
grad2 = []  # Gradient 2 values

# Main loop for calculating gradient and cost for 100 iterations
for i in range(100):
    # Compute perturbed cost function
    cost2 = (x1 + 0.1 * np.sin(wc1 * t) - 5) ** 2 + (x2 + 0.1 * np.sin(wc2 * t) - 3) ** 2

    # Define filter coefficients
    b = [1, 0, -1]
    a1 = [1, -np.exp(-28.2743 + 564.779 * 1j) - np.exp(-28.2743 - 564.779 * 1j), np.exp(-28.2743 * 2)]
    a2 = [1, -np.exp(-31.4159 + 627.533 * 1j) - np.exp(-31.4159 - 627.533 * 1j), np.exp(-31.4159 * 2)]

    # Apply filters to the perturbed cost function and take the real parts
    y01 = np.real(filter(b, a1, cost2))
    y02 = np.real(filter(b, a2, cost2))

    # Multiply by sin(wc*t)
    y11 = 0.1 * np.sin(wc1 * t) * y01
    y12 = 0.1 * np.sin(wc2 * t) * y02

    # Define coefficients for secondary filter
    b_filter = [0.55 * (1 - np.exp(-100 * np.pi)) ** 2 * 2 / 0.1 ** 2]
    a_filter = [1, -2 * np.exp(-100 * np.pi), np.exp(-200 * np.pi)]

    # Apply secondary filter and take the real parts
    y21 = np.real(filter(b_filter, a_filter, y11))
    y22 = np.real(filter(b_filter, a_filter, y12))

    # Update x1 and x2 values
    x1 = x1 - 0.05 * np.mean(y21[24:75])
    x2 = x2 - 0.05 * np.mean(y22[24:75])

    # Recompute cost and update lists
    x.append(i)
    cost = (x1 - 5) ** 2 + (x2 - 3) ** 2
    y.append(cost)

    # Compute difference and gradients
    diff = 2 * (x1 - 5) + 2 * (x2 - 3)
    grad1.append([diff, np.mean(y21[24:75] + y22[24:75])])
    grad2.append(grad1[i][0] - grad1[i][1])

# Plotting gradient values using Plotly
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=x, y=[g[0] for g in grad1], mode='lines', name='grad1_1'))
fig1.add_trace(go.Scatter(x=x, y=[g[1] for g in grad1], mode='lines', name='grad1_2'))
fig1.update_layout(
    title="Gradient Values Over Iterations",
    xaxis_title="Iteration",
    yaxis_title="Gradient Value"
)
fig1.show()

# Plotting cost values using Plotly
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=list(range(len(y))), y=y, mode='lines'))
fig2.update_layout(
    title="Cost Values Over Iterations",
    xaxis_title="Iteration",
    yaxis_title="Cost Value"
)
fig2.show()

# Print mean of the summed filtered values
print(np.mean(y21[24:75] + y22[24:75]))


-0.05306029126098556
